In [1]:
import numpy as np
from implementations_clean import *
from proj1_helpers import *

In [2]:
y,X,ids = load_csv_data("train.csv")

In [3]:
import pandas as pd
X = np.where(X == -999., np.nan, X)
df = pd.DataFrame(X)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2.0,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,160.937,68.768,103.235,48.146,NaN,NaN,NaN,3.473,2.078,125.157,...,-1.916,164.546,1.0,46.226,0.725,1.158,NaN,NaN,NaN,46.226
2,NaN,162.172,125.953,35.635,NaN,NaN,NaN,3.148,9.336,197.814,...,-2.186,260.414,1.0,44.251,2.053,-2.028,NaN,NaN,NaN,44.251
3,143.905,81.417,80.943,0.414,NaN,NaN,NaN,3.310,0.414,75.968,...,0.060,86.062,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000
4,175.864,16.915,134.805,16.405,NaN,NaN,NaN,3.891,16.405,57.983,...,-0.871,53.131,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000


In [4]:
#feature 0 replace nan with average (and add boolean for nan?)
col_means = np.nanmean(X, axis=0)
idxs = np.where(np.isnan(X))
X[idxs] = np.take(col_means, idxs[1])

#feature 1: correlations der_mass_MMC
co = 130
X_gt_mmc = np.array(X[:,0], copy=True)
X_gt_mmc[X_gt_mmc <= co] = co
# X = np.column_stack((X, X_gt_mmc))
X[:,0][X[:,0] > co] = co
X = np.column_stack((X, X_gt_mmc))

#feature 2: add momentums
#tau momentum
tau_px = X[:,13]*np.cos(X[:,15])
tau_py = X[:,13]*np.sin(X[:,15])
tau_pz = X[:,13]*np.sinh(X[:,14])
X = np.column_stack((X, tau_px,tau_py,tau_pz))
#lep momentum
lep_px = X[:,16]*np.cos(X[:,18])
lep_py = X[:,16]*np.cos(X[:,18])
lep_pz = X[:,16]*np.cos(X[:,17])
X = np.column_stack((X, lep_px,lep_py,lep_pz))
#leading jet momentum
jet_px = X[:,22]*np.cos(X[:,24])
jet_py = X[:,22]*np.cos(X[:,24])
jet_pz = X[:,22]*np.cos(X[:,23])
X = np.column_stack((X, jet_px,jet_py,jet_pz))
#subleading jet momentum
subjet_px = X[:,25]*np.cos(X[:,27])
subjet_py = X[:,25]*np.cos(X[:,27])
subjet_pz = X[:,25]*np.cos(X[:,26])
X = np.column_stack((X, subjet_px,subjet_py,subjet_pz))

# feature 3: abs angles
#der_met_phi_centrality
X[:,11] = np.abs(X[:,11])
#tau phi
X[:,15] = np.abs(X[:,15])
#lep phi
X[:,18] = np.abs(X[:,18])
#met phi
X[:,20] = np.abs(X[:,20])
#lead jet phi
X[:,24] = np.abs(X[:,24])
#sublead jet phi
X[:,27] = np.abs(X[:,27])

df = pd.DataFrame(X)
df.head(20)
X[:,11].mean()

1.1387578679999997

In [5]:
X[:,11].min()

0.0

In [6]:
def make_features(X):
    # converting -999. to nan to use np.nanmean and np.nanstd
    X = np.where(X == -999., np.nan, X)
    # standardizing the data Xd = (X_d - E[X_d])/(std(X_d))
    X, means, stds = standardize(X)
    # since data is standirdized, the mean is more or less 0 for each feature so replacing by zero is reasonable and helps computations
    X = np.where(np.isnan(X), 0, X)
    # adding the 1 padding
    return np.column_stack((np.ones(X.shape[0]), X))

In [7]:
# X,_,_ = standardize(X)
# bias = np.ones((X.shape[0],1))
# X = np.hstack([bias,X])
X = make_features(X)

In [8]:
df = pd.DataFrame(X)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,1.0,0.925210,0.068332,0.407680,-0.469966,-1.591638e+00,-1.153306e+00,1.806346e+00,0.882478,1.033099,...,0.480884,-1.047625,-1.047625,-1.767430,-1.267320,-1.267320,-0.238641,0.122568,0.122568,-0.256095
1,1.0,0.925210,0.552505,0.540136,-0.153167,-7.229012e-12,-6.304978e-12,6.751165e-13,1.404888,-0.756027,...,1.922023,0.997382,0.997382,0.463829,0.645996,0.645996,-0.652951,0.945345,0.945345,0.614340
2,1.0,0.555891,3.195156,1.096560,-0.349710,-7.229012e-12,-6.304978e-12,6.751165e-13,0.989770,-0.430168,...,-0.289035,1.635168,1.635168,1.751947,-0.612340,-0.612340,0.970068,-1.640389,-1.640389,-1.050357
3,1.0,0.925210,0.910379,-0.005853,-0.903016,-7.229012e-12,-6.304978e-12,6.751165e-13,1.196690,-0.830735,...,-0.681156,-1.447571,-1.447571,0.933633,-0.130971,-0.130971,-0.015695,-0.004537,-0.004537,0.002805
4,1.0,0.925210,-0.914556,1.313369,-0.651804,-7.229012e-12,-6.304978e-12,6.751165e-13,1.938794,-0.112795,...,-1.504646,0.001200,0.001200,0.070223,-0.130971,-0.130971,-0.015695,-0.004537,-0.004537,0.002805


In [9]:
# np.random.shuffle(X)
cutoff = int(0.8*((X.shape)[0]))
X_train = X[:cutoff]
y_train = y[:cutoff]
X_test = X[cutoff:]
y_test = y[cutoff:]

In [10]:
initial_w = np.zeros(X.shape[1])
# max_iters = 10*200000
max_iters = 5000
gamma = 0.1
lambda_ = 0.001

In [11]:
print(initial_w)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
# loss,w = GD_logistic_regression(y = y_train, tx = X_train, initial_w = initial_w,
#                              max_iters = max_iters, gamma = gamma)
loss,w = GD_reg_logistic_regression(y = y_train, tx = X_train, initial_w = initial_w,
                             max_iters = max_iters, gamma = gamma, lambda_ = lambda_)
# loss,w = logistic_regression(y = y_train, tx = X_train, initial_w = initial_w,
#                              max_iters = max_iters, gamma = gamma)
# loss,w = least_squares_GD(y = y_train,tx = X_train, initial_w = initial_w, max_iters = max_iters, gamma = gamma)

Iteration:100, loss : 0.476663463146636
Iteration:200, loss : 0.4598102229287025
Iteration:300, loss : 0.4539913079189095
Iteration:400, loss : 0.4511144012042898
Iteration:500, loss : 0.44940522447628006
Iteration:600, loss : 0.4482748653965195
Iteration:700, loss : 0.44747664301612655
Iteration:800, loss : 0.4468884839971508
Iteration:900, loss : 0.4464421214815062
Iteration:1000, loss : 0.44609601451288533
Iteration:1100, loss : 0.44582325381167043
Iteration:1200, loss : 0.4456055544171438
Iteration:1300, loss : 0.44543002692584993
Iteration:1400, loss : 0.4452873205465378
Iteration:1500, loss : 0.44517049381324564
Iteration:1600, loss : 0.4450742959755394
Iteration:1700, loss : 0.44499469246886764
Iteration:1800, loss : 0.4449285418404589
Iteration:1900, loss : 0.4448733701100977
Iteration:2000, loss : 0.4448272097423477
Iteration:2100, loss : 0.444788482574738
Iteration:2200, loss : 0.44475591330952974
Iteration:2300, loss : 0.44472846466245103
Iteration:2400, loss : 0.44470528811

In [13]:
#train accuracy
y_pred = classification(X_train @ w)
acc = 1-np.sum(np.abs(y_pred - y_train)) / X_train.shape[0]
print(acc)

0.79357


In [14]:
#test accuracy
y_pred = classification(X_test @ w)
acc = 1-np.sum(np.abs(y_pred - y_test)) / X_test.shape[0]

In [15]:
acc

0.79726

In [16]:
# y,X_sub,ids = load_csv_data("test.csv")
# #feature 1: correlations der_mass_MMC
# col_means = np.nanmean(X_sub, axis=0)
# idxs = np.where(np.isnan(X_sub))
# X_sub[idxs] = np.take(col_means, idxs[1])
# X_gt_mmc = np.array(X_sub[:,0], copy=True)
# X_gt_mmc[X_gt_mmc <= 140] = 140
# # X = np.column_stack((X, X_gt_mmc))
# X_sub[:,0][X_sub[:,0] > 140] = 140
# X_sub = np.column_stack((X_sub, X_gt_mmc))

# #feature 2: add momentums
# #tau momentum
# tau_px = X_sub[:,13]*np.cos(X_sub[:,15])
# tau_py = X_sub[:,13]*np.sin(X_sub[:,15])
# tau_pz = X_sub[:,13]*np.sinh(X_sub[:,14])
# X_sub = np.column_stack((X_sub, tau_px,tau_py,tau_pz))
# #lep momentum
# lep_px = X_sub[:,16]*np.cos(X_sub[:,18])
# lep_py = X_sub[:,16]*np.cos(X_sub[:,18])
# lep_pz = X_sub[:,16]*np.cos(X_sub[:,17])
# X_sub = np.column_stack((X_sub, lep_px,lep_py,lep_pz))
# #leading jet momentum
# jet_px = X_sub[:,22]*np.cos(X_sub[:,24])
# jet_py = X_sub[:,22]*np.cos(X_sub[:,24])
# jet_pz = X_sub[:,22]*np.cos(X_sub[:,23])
# X_sub = np.column_stack((X_sub, jet_px,jet_py,jet_pz))
# #subleading jet momentum
# subjet_px = X_sub[:,25]*np.cos(X_sub[:,27])
# subjet_py = X_sub[:,25]*np.cos(X_sub[:,27])
# subjet_pz = X_sub[:,25]*np.cos(X_sub[:,26])
# X_sub = np.column_stack((X_sub, subjet_px,subjet_py,subjet_pz))

# # feature 3: abs angles
# #der_met_phi_centrality
# X_sub[:,11] = np.abs(X_sub[:,11])
# #tau phi
# X_sub[:,15] = np.abs(X_sub[:,15])
# #lep phi
# X_sub[:,18] = np.abs(X_sub[:,18])
# #met phi
# X_sub[:,20] = np.abs(X_sub[:,20])
# #lead jet phi
# X_sub[:,24] = np.abs(X_sub[:,24])
# #sublead jet phi
# X_sub[:,27] = np.abs(X_sub[:,27])

# X_sub = make_features(X_sub)

In [17]:
# sub_pred = predict_labels(w,X_sub) 
# sub_pred.mean()

In [18]:
# create_csv_submission(ids, sub_pred, "submission_test.csv")
# t = pd.read_csv("submission_test.csv")
# t.to_csv("submission_test.csv",index = False)